In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys

from scipy.stats import chi2_contingency
from statsmodels.stats.power import GofChisquarePower

# Add src/ to path for custom module imports
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root / "src"))

from paths import CLEAN_DATA_DIR

In [2]:
cleaned_data = pd.read_parquet(CLEAN_DATA_DIR / "cleaned_experiments.parquet")

### Our challenge

We have a series of 2 level tests - those are a simple two-proportion Z-test and corresponding power calculation. We also have 3+ level tests. Those need a chi-squared contingency table. We want the power for that as well.

#### Pairwise Z-tests and Multiple Comparisons
We could look at the 3+ levels and run pairwise two-prop Z-tests, but that is unwise. More experiments = more errors. There are no benefits to multiple Z-tests, so we will not do that. Nobody should ever do multiple Z-tests fo the same factor levels. It is a poor decision every time.

#### Keeping it simple
For a test with 2 levels, the Chi-Squared and two-prop Z-tests yield identical results. The two-prop Z-tests only work two levels - they cannot adapt to 3+ levels. As such, we are only going to use chi-squared tests. This will keep the code simple without compromising our results.

#### Power analysis
We will used the observed results and Cohen's *w*. Usually we do a power analysis before the tests, but this is multiple years old data and I don't have the capacity to determine the necessary N for a given effect size. As such, we use Cohen's *w* as it is most logical in this case.

#### The goal

We need a function that will perform a statistical test, identify any winning headlines, and return the power associated with the test. Let's get to it.

In [3]:
## start by making a copy
df = cleaned_data.copy()

In [4]:
## put these into groups
grouped = df.groupby("clickability_test_id")

## get the important data from grouped
for test_id, group in grouped:
    group = group.copy()
    n_variants = group.shape[0]

In [5]:
## this appears as expected
print(group)
print(n_variants)

          clickability_test_id  \
8639  55413ef4333531000c180000   
8640  55413ef4333531000c180000   

                                               headline  impressions  clicks  \
8639  They spent 4 days in the swamp looking for a c...         2091      32   
8640  They spent 4 days in the swamp looking for a m...         2014      18   

      proportion  
8639    0.015304  
8640    0.008937  
2


In [6]:
# add in some logic that skips anything with less than 2 (shouldn't exist)
# also skip anything with 0 impressions (shouldn't exist either)

## this is commented out because we are not in the loop, but remember to add it later
#if n_variants < 2 or (group['impressions'] == 0).any():
#    continue

impressions = group['impressions'].values
clicks = group['clicks'].values
ctr = group['proportion'].values

In [7]:
## create the contingency table
data = np.vstack([clicks, impressions - clicks]).T

In [8]:
data

array([[  32, 2059],
       [  18, 1996]])

In [9]:
chi2, pval, dof, expected = chi2_contingency(data)

If we reject the null, pick the highest CTR, else, no winner

In [10]:
## If we reject, identify winner as highest pval
winner = None
if pval < 0.05:
    best_idx = np.argmax(ctr)
    winner = group.iloc[best_idx]['headline']

In [11]:
# power estimate for these data
total_n = impressions.sum()

In [12]:
print(winner)

None


In [13]:
## Compute observed effect size (Cohen's w)
effect_size = np.sqrt(chi2 / total_n)

## define this upfront in the function
power_calc = GofChisquarePower()

## Estimate power using observed effect size
power = power_calc.power(effect_size=effect_size, nobs=total_n, alpha=0.05, n_bins=n_variants)

print(power)

0.40399074353086906


We have a no-winner, low power test. This is a statistician's nightmare. It leads to questions like "why did we bother?" or "Did we set up the experiment correctly?"

### Seting up tests matters: do not skip your sample size calculation

The big takeaway from the test we performed above is that there was *no point in running the experiment.* Power = 0.40 means we had a 40% chance of detecting a true effect if we had rejected the null. We failed to reject the null, which is a valid outcome, but even if we had then it's more likely than not we rejected the null incorrectly. Leading to my statement of *"why bother?"*

#### Getting it right - how I would have worked with my business partners

I would have engaged with my business partners to help them understand both components of a statistical test and define the necessary sample sizes. That way, we could get valid results to help make better decisions in the long-term. It's possible that changing headlines is irrelevant - but we need a valid test to confirm that fact. Without the valid test we cannot draw those conclusions. It leads to two primary issues, in my mind:

1) Wasted resources
    - A poorly defined test wastes time and money. Don't bother doing it if you don't commit to doing it correctly
2) Incorrect conclusions lead to implementing inefficiencies
    - If we draw conclusions based on poorly defined tests, then we risk implementing incorrect procedures. Our "best practices" may be immaterial. The business becomes less efficient overall, makes more work for itself, or fails to capitalize on opportunities because of poor tests.

In [14]:
def analyze_ab_tests(df, alpha=0.05):
    results = []

    grouped = df.groupby("clickability_test_id")
    power_calc = GofChisquarePower()

    for test_id, group in grouped:
        group = group.copy()
        n_variants = group.shape[0]

        ## these should not appear, but let's handle if they do
        if n_variants < 2 or (group['impressions'] == 0).any():
            continue

        ### retriece results from the tests
        impressions = group['impressions'].values
        clicks = group['clicks'].values
        ctr = group['proportion'].values

        ## set up the contingency table
        data = np.vstack([clicks, impressions - clicks]).T

        ## get overall N
        total_n = impressions.sum()

        try:
            chi2, pval, dof, expected = chi2_contingency(data)

            ## get the effect size
            effect_size = np.sqrt(chi2 / total_n)

            ## get our power estimate
            power = power_calc.power(effect_size=effect_size, nobs=total_n, alpha=alpha, n_bins=n_variants)

        except Exception:
            pval = np.nan
            effect_size = np.nan
            power = np.nan

        winner = None
        if pval < alpha:
            best_idx = np.argmax(ctr)
            winner = group.iloc[best_idx]['headline']

        results.append({
            "test_id": test_id,
            "n_variants": n_variants,
            "p_value": pval,
            "winner": winner,
            "power_estimate": power,
            "effect_size": effect_size
        })

    return pd.DataFrame(results)


In [15]:
results = analyze_ab_tests(df, alpha=0.05)

In [16]:
print(results.shape)
results[(results['p_value'] <= 0.05) & (results['power_estimate'] >= .8)]

(1951, 6)


,test_id,n_variants,p_value,winner,power_estimate,effect_size
1,51436069220cb800020005ae,5,2.029175e-05,"The Advertisements You Read Every Day, Only Naked",0.993562,0.052454
3,5143606a220cb800020005c6,4,5.350506e-04,Her Mom Abandoned Her When She Found Out She H...,0.954769,0.039843
4,5143606b220cb800020005d7,5,2.746557e-06,Feminist Confuses Fox News Host By Suggesting ...,0.997939,0.053134
8,51436071220cb80002000788,4,1.198781e-03,How To Stop A Child From Smoking In 30 Seconds,0.932257,0.038705
9,51436073220cb8000200079c,4,4.771824e-03,7 Love Letters To Learning From The World's Sh...,0.869021,0.034417
...,...,...,...,...,...,...
1933,5537fb95666331002c230000,6,1.259798e-02,"Sweet, naive Amy Schumer stumbles upon 3 show ...",0.853765,0.030924
1934,553825c7666331002c710300,2,8.108550e-04,"John Oliver explains the totally slimy, comple...",0.917599,0.029670
1935,553950e2353130000ce20000,6,1.095320e-09,The things a black kid probably can't do that ...,0.999986,0.064464
1944,553fb55b616464000cfc0100,6,2.673924e-03,He tells the story of what 2 neo-Nazis did to ...,0.930329,0.038638


We have 778 tests with a small p-value and large power based on the observed data.

In [17]:
results[(results['power_estimate'] <= .8)]

,test_id,n_variants,p_value,winner,power_estimate,effect_size
0,51436061220cb800020001e7,4,0.039912,The One Where A Creationist Picks A Fight And ...,0.672974,0.028305
2,51436069220cb800020005bd,2,0.250428,None,0.209720,0.011177
5,5143606e220cb800020006ba,4,0.029888,This Woman's Beef With Prettiness Will Leave Y...,0.708869,0.028648
6,51436070220cb80002000748,3,0.188788,None,0.353352,0.017654
7,51436071220cb8000200077c,4,0.023038,What Are The Odds That You'll Be Single Foreve...,0.738437,0.030084
...,...,...,...,...,...,...
1945,553fbd5e356133001c050000,4,0.051261,None,0.639238,0.030825
1946,553fd64a356133000c450100,6,0.254789,None,0.471741,0.018970
1947,553fdd3c316638000c800000,5,0.486569,None,0.277603,0.013719
1949,55403be0393131002cf60000,4,0.033667,"A plastic bottle finds its way home, but its o...",0.694489,0.032603


We have 1173 tests with power less than 0.80. Those are tests we probably should not have run in the first place.

In [18]:
results[(results['p_value'] <= 0.05) & (results['power_estimate'] <= .8)]

,test_id,n_variants,p_value,winner,power_estimate,effect_size
0,51436061220cb800020001e7,4,0.039912,The One Where A Creationist Picks A Fight And ...,0.672974,0.028305
5,5143606e220cb800020006ba,4,0.029888,This Woman's Beef With Prettiness Will Leave Y...,0.708869,0.028648
7,51436071220cb8000200077c,4,0.023038,What Are The Odds That You'll Be Single Foreve...,0.738437,0.030084
14,51436075220cb800020007b8,4,0.031017,Someone Give This Man A Nobel Prize Already. H...,0.704450,0.027303
26,51436081220cb80002000981,4,0.034670,China's Ivory Consumption Will Probably Kill E...,0.690858,0.027856
...,...,...,...,...,...,...
1924,553689b2346161000c420100,4,0.016238,Canceling out the harmful effects of sitting c...,0.774283,0.035548
1925,55368f0c346161001c740100,6,0.027096,A major American theater just cast a black wom...,0.792854,0.029805
1942,553faeb2336438001c7f0100,6,0.033854,Ever heard of the Battle of Blair Mountain? Th...,0.771357,0.031422
1943,553fb407616464002cd40100,4,0.017864,There's a simple fix to save the life of an ov...,0.764929,0.035225


There are 231 tests where we risk drawing a false conclusion. There is a small p-val, but the test is underpowered. This is *extremely dangerous* and should make most statisticians uncomfortable. It looks like a few might be close to .80 - I would be flexible here, but it's still dangerous.